In [1]:
import os

from importlib.metadata import version

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

XRADIO version 0.0.16 already installed.


# Download ms

In [2]:
import graphviper

graphviper.utils.data.download(file="Antennae_North.cal.lsrk.split.ms")

[2024-02-07 12:01:11,120]     INFO  graphviper:  File exists: Antennae_North.cal.lsrk.split.ms 


# Convert ms

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

main_chunksize = {"frequency": 3}
in_file = "Antennae_North.cal.lsrk.split.ms"
out_file = "Antennae_North.cal.lsrk.split.vis.zarr"
convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

# Lazy Read PS

In [4]:
from xradio.vis.read_processing_set import read_processing_set

# sel_parms = {'xds_id':[0,1,2]}
intents = ["OBSERVE_TARGET#ON_SOURCE"]
ps = read_processing_set(out_file, intents=intents)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency
0,Antennae_North.cal.lsrk.split_ddi_0_intent_OBS...,0,OBSERVE_TARGET#ON_SOURCE,0,NGC4038 - Antennae North,3.439281e+11,3.440067e+11
1,Antennae_North.cal.lsrk.split_ddi_0_intent_OBS...,0,OBSERVE_TARGET#ON_SOURCE,2,NGC4038 - Antennae North,3.439281e+11,3.440067e+11
2,Antennae_North.cal.lsrk.split_ddi_0_intent_OBS...,0,OBSERVE_TARGET#ON_SOURCE,1,NGC4038 - Antennae North,3.439281e+11,3.440067e+11


# A single ms v4

In [5]:
ps["Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1"]

<xarray.Dataset>
Dimensions:                     (time: 35, baseline_id: 64, frequency: 8,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 dask.array<chunksize=(64,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 dask.array<chunksize=(64,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 0 1 2 3 4 ... 59 60 61 62 63
  * frequency                   (frequency) float64 3.439e+11 ... 3.44e+11
  * polarization                (polarization) <U2 'XX' 'YY'
  * time                        (time) float64 1.307e+09 1.307e+09 ... 1.307e+09
  * uvw_label                   (uvw_label) <U1 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 dask.array<chunksize=(35, 64), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool dask.array<chunksize=(35, 64, 3, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 dask.array<chunksize=(35, 64), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 dask.array<chunksize=(35, 64, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 dask.array<chunksize=(35, 64, 3, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 dask.array<chunksize=(35, 64, 3, 2), meta=np.ndarray>
Attributes:
    data_groups:  {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'VISI...
    ddi:          0
    field_info:   {'code': 'none', 'delay_direction': {'attrs': {'frame': 'FK...
    intent:       OBSERVE_TARGET#ON_SOURCE
    antenna_xds:  <xarray.Dataset>\nDimensions:        (antenna_id: 15, xyz_l...

## Attributes of Data Arrays and Coordinates

All **data arrays** and **coordinates** can have measures information in their attributes section along with other relevant metadata. 

List of measures:
https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1504318014

Measures dictionary format (based on https://docs.xarray.dev/en/stable/generated/xarray.DataArray.from_dict.html):
{"dims":..., "data":..., "attrs":measures_dict} 


## Frequency Coordinate Example

The ```frequency``` coordinate is a spectral coord measure (keys: type,units,reference_frame) but also contains the channel_width, effective_channel_width, spectral_window_name, and reference_frequency. Any metadata that is a non-id number should be in the relevant measures dictionary (for example channel_width and reference_frequency.) 

In [6]:
ps[
    "Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1"
].frequency

<xarray.DataArray 'frequency' (frequency: 8)>
array([3.439281e+11, 3.439393e+11, 3.439506e+11, 3.439618e+11, 3.439730e+11,
       3.439843e+11, 3.439955e+11, 3.440067e+11])
Coordinates:
  * frequency  (frequency) float64 3.439e+11 3.439e+11 ... 3.44e+11 3.44e+11
Attributes:
    channel_width:         {'attrs': {'type': 'quantity', 'units': ['Hz']}, '...
    frame:                 LSRK
    reference_frequency:   {'attrs': {'frame': 'LSRK', 'type': 'spectral_coor...
    spectral_window_name:  
    spw_id:                0
    type:                  spectral_coord
    units:                 ['Hz']

In [7]:
import json

print(
    json.dumps(
        ps[
            "Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1"
        ].frequency.attrs,
        indent=4,
    )
)

{
    "channel_width": {
        "attrs": {
            "type": "quantity",
            "units": [
                "Hz"
            ]
        },
        "data": 11231488.981445312,
        "dims": ""
    },
    "frame": "LSRK",
    "reference_frequency": {
        "attrs": {
            "frame": "LSRK",
            "type": "spectral_coord",
            "units": [
                "Hz"
            ]
        },
        "data": 343928096685.9587,
        "dims": ""
    },
    "spectral_window_name": "",
    "spw_id": 0,
    "type": "spectral_coord",
    "units": [
        "Hz"
    ]
}


## Time Example
Another example is the time coordinate that is a time measure (keys: type, units, time_scale, format) but also contains integration_time and effective_integration_time.

In [8]:
import json

print(
    json.dumps(
        ps[
            "Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1"
        ].time.attrs,
        indent=4,
    )
)

{
    "effective_integration_time": "EFFECTIVE_INTEGRATION_TIME",
    "format": "unix",
    "integration_time": {
        "attrs": {
            "type": "quantity",
            "units": [
                "s"
            ]
        },
        "data": 6.048,
        "dims": ""
    },
    "scale": "UTC",
    "type": "time",
    "units": [
        "s"
    ]
}


## Info Attributes

The ms v4 also allows for info dictionaries in the attribute section of the dataset (used when data is not n-dimensional). Just like the non-id number metadata in coordinates and data arrays the relevant measure should be used. An example is the field_info where the delay_direction, phase_direction, and reference_direction are stored as sky_coord measures (keys: type, units, reference_frame).

In [9]:
import json

print(
    json.dumps(
        ps[
            "Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1"
        ].field_info,
        indent=4,
    )
)

{
    "code": "none",
    "delay_direction": {
        "attrs": {
            "frame": "FK5",
            "type": "sky_coord",
            "units": [
                "rad",
                "rad"
            ]
        },
        "data": [
            3.149807242890337,
            -0.3292996087690852
        ],
        "dims": ""
    },
    "field_id": 1,
    "name": "NGC4038 - Antennae North",
    "phase_direction": {
        "attrs": {
            "frame": "FK5",
            "type": "sky_coord",
            "units": [
                "rad",
                "rad"
            ]
        },
        "data": [
            3.149807242890337,
            -0.3292996087690852
        ],
        "dims": ""
    },
    "reference_direction": {
        "attrs": {
            "frame": "FK5",
            "type": "sky_coord",
            "units": [
                "rad",
                "rad"
            ]
        },
        "data": [
            3.149807242890337,
            -0.3292996087690852
    

## Antenna Dataset

In [10]:
ant_xds = ps[
    "Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1"
].attrs["antenna_xds"]
ant_xds

<xarray.Dataset>
Dimensions:        (antenna_id: 15, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
    mount          (antenna_id) <U6 dask.array<chunksize=(15,), meta=np.ndarray>
    name           (antenna_id) <U4 dask.array<chunksize=(15,), meta=np.ndarray>
    station        (antenna_id) <U4 dask.array<chunksize=(15,), meta=np.ndarray>
    type           (antenna_id) <U12 dask.array<chunksize=(15,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 dask.array<chunksize=(15,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 dask.array<chunksize=(15, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 dask.array<chunksize=(15, 3), meta=np.ndarray>

In [11]:
ant_xds.station.values

array(['A015', 'J505', 'T704', 'A004', 'A072', 'A008', 'A009', 'A016',
       'T702', 'A017', 'J504', 'A067', 'A011', 'A137', 'A075'],
      dtype='<U4')